<a href="https://colab.research.google.com/github/hyeonji0401/CUDA_practice/blob/main/thread_hierarchy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4-1. CUDA 스레드 계층 구조

**1. CUDA 스레드 계층**
- 스레드 -> 워프 -> 블록 -> 그리드로 이루어짐

1) 스레드
- 계층 구조에서 가장 작은 단위
- CUDA 연산을 수행하거나 CUDA 코어르 사용하는 기본 단위
- 커널 코드는 모든 스레드에 공우되며 각 스레드가 독립적으로 커널 코드를 수행

2) 워프
- 계층 구조의 두번째 계층
- 32개의 스레드를 하나로 묶은 것을 말하며 CUDA의 기본 수행단위
- **기본수행단위 :** 한 워프에 속한 스레드들은 하나의 제어 장치에 의해 제어됨
- GPU의 SIMT구조에서 멀티 스레드의 단위가 되는 것이 워프임
=> 즉 하나의 명령에 따라 32개의 스레드가 동시에 움직임

3) 블록(스레드 블록)
- 워프보다 상위 스레드 그룹으로 워프들의 집합을 의미함
- 하나의 블록에 포함된 각 스레드는 자신만의 고유한 스레드 번호(thread ID)를 가짐

  : 동일한 블록 안에는 동일한 번호를 갖는 스레드가 없음
- 각 블록은 자신만의 고유한 블록 번호(block ID)가 있음

=> 원하는 스레드를 지정하기 위해서는 블록 번호와 스레드 번호를 모두 사용해야함

  : 블록 내 스레드는 최대 3차원 번호를 가질 수 있음

4) 그리드
- 계층 구조에서 가장 상위 단계
- 여러 개의 블록을 포함하는 블록들의 그룹
- 하나의 그리드에 포함된 블록들은 서로 다른 자신만의 번호를 가지며 그리드 내 블록 또한 최대 3차원 형태로 배치될 수 있음
- 커널이 호츨되면 그리드가 생성됨

=> 하나의 그리드는 하나의 커널 호출과 1:1 대응되게 해당 커널을 수행한 스레드를 생성함
